# 감귤 착과량 예측 AI 경진대회

### https://dacon.io/codeshare/7298

In [70]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor
import numpy as np

df = pd.read_csv('./ensemble/train.csv') # train 데이터 path

X_train = df.drop(['ID','착과량(int)'], axis=1)
y_train = df['착과량(int)']

### TrainData

In [71]:
col_saesoon = X_train.iloc[:, 4:93] #새순 2022~09-01 ~ 
col_yeoprok = X_train.iloc[:, 93:]  # 엽록소 2022~09~01 

saesoon_mean = pd.DataFrame()
for i in col_saesoon.iterrows():
    temp = pd.DataFrame()
    for o in range(5, len(i[1]), 5):
        temp = pd.concat([temp, pd.Series(i[1][o-5:o].mean())], ignore_index=True, axis=1) #5일 간격의 평균 새순 길이의 평균 
    temp = pd.concat([temp, pd.Series(i[1][85:].mean())], ignore_index=True, axis=1) # 나머지 처리 
    saesoon_mean = saesoon_mean.append(temp)

saesoon_mean = saesoon_mean.reset_index(drop=True)

yeoprok_mean = pd.DataFrame()
for i in col_yeoprok.iterrows():
    temp = pd.DataFrame()
    for o in range(5, len(i[1]), 5):
        temp = pd.concat([temp, pd.Series(i[1][o-5:o].mean())], ignore_index=True, axis=1) #5일 간격의 평균 엽록소 
    temp = pd.concat([temp, pd.Series(i[1][85:].mean())], ignore_index=True, axis=1) #나머지 처리 
    yeoprok_mean = yeoprok_mean.append(temp)

yeoprok_mean = yeoprok_mean.reset_index(drop=True)
yeoprok_mean = yeoprok_mean.div(X_train['수고(m)'], axis=0) #5일 간의 평균 엽록소 값에 대해 수고로 나누어줌

X_train_new = pd.concat([X_train.iloc[:, :4], saesoon_mean, yeoprok_mean], ignore_index=True, axis=1) #기존의 수고, 수관폭1, 수관폭2, 수관폭 평균과함께 새로운 dataframe을 생성하였습니다. 
X_train_new['cntzero'] = col_saesoon.apply(lambda x: x[x.isin([0.000])].count(), axis=1) #새순 feature가 0일 수록 낮은 착과량을 보이기 때문에 -< 새순이 0인날을 집계하여 파생변수로 생성함. 

/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/152460346.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  saesoon_mean = saesoon_mean.append(temp)
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/152460346.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  saesoon_mean = saesoon_mean.append(temp)
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/152460346.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  saesoon_mean = saesoon_mean.append(temp)
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/152460346.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  saesoon_mean 

### TestData(: train data와 전처리 방식 동일)

In [75]:
test = pd.read_csv('./ensemble/test.csv') # test 데이터 path
test.drop(['ID'], axis=1, inplace=True)

tcol_saesoon = test.iloc[:, 4:93]
tcol_yeoprok = test.iloc[:, 93:]


tsaesoon_mean = pd.DataFrame()
for i in tcol_saesoon.iterrows():
    temp = pd.DataFrame()
    for o in range(5, len(i[1]), 5):
        temp = pd.concat([temp, pd.Series(i[1][o-5:o].mean())], ignore_index=True, axis=1)
    temp = pd.concat([temp, pd.Series(i[1][85:].mean())], ignore_index=True, axis=1)
    tsaesoon_mean = tsaesoon_mean.append(temp)

tsaesoon_mean = tsaesoon_mean.reset_index(drop=True)

tyeoprok_mean = pd.DataFrame()
for i in tcol_yeoprok.iterrows():
    temp = pd.DataFrame()
    for o in range(5, len(i[1]), 5):
        temp = pd.concat([temp, pd.Series(i[1][o-5:o].mean())], ignore_index=True, axis=1)
    temp = pd.concat([temp, pd.Series(i[1][85:].mean())], ignore_index=True, axis=1)
    tyeoprok_mean = tyeoprok_mean.append(temp)

tyeoprok_mean = tyeoprok_mean.reset_index(drop=True)
tyeoprok_mean = tyeoprok_mean.div(test['수고(m)'], axis=0)

test_new = pd.concat([test.iloc[:, :4], tsaesoon_mean, tyeoprok_mean], ignore_index=True, axis=1)
test_new['cntzero'] = tcol_saesoon.apply(lambda x: x[x.isin([0.000])].count(), axis=1)


/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/3936329022.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tsaesoon_mean = tsaesoon_mean.append(temp)
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/3936329022.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tsaesoon_mean = tsaesoon_mean.append(temp)
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/3936329022.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tsaesoon_mean = tsaesoon_mean.append(temp)
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_6118/3936329022.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tsa

In [76]:
X_train_new.columns = X_train_new.columns.astype(str)
test_new.columns = test_new.columns.astype(str)

### 모델 정의
- ExtraTreeRegressor와 RandomForest Regressor ensemble
- voiting Regressor를 통해 앙상블할 classifier를 튜플형식으로 넣어줌
- 해당 코드에서는 voting 방식은 hard(default)

In [73]:
et_reg = ExtraTreesRegressor(n_estimators=1000, min_samples_leaf=9, min_samples_split=6, max_features=40)
rf_reg = RandomForestRegressor(n_estimators=1000, min_samples_leaf=9, min_samples_split=6, max_features=20)

voting_reg = VotingRegressor(
    estimators=[
        ('et', et_reg),
        ('rf', rf_reg)
    ]
)
voting_reg.fit(X_train_new, y_train)

VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(max_features=40,
                                                 min_samples_leaf=9,
                                                 min_samples_split=6,
                                                 n_estimators=1000)),
                            ('rf',
                             RandomForestRegressor(max_features=20,
                                                   min_samples_leaf=9,
                                                   min_samples_split=6,
                                                   n_estimators=1000))])

In [78]:
pred = voting_reg.predict(test_new)
sample_submission = pd.read_csv('./ensemble/sample_submission.csv')
sample_submission['착과량(int)'] = pred
sample_submission.to_csv("./submit.csv", index = False)

In [79]:
sample_submission.head(10)

,ID,착과량(int)
0,TEST_0000,244.772718
1,TEST_0001,749.912617
2,TEST_0002,153.900840
3,TEST_0003,445.992485
4,TEST_0004,677.112642
5,TEST_0005,276.183710
6,TEST_0006,76.150599
7,TEST_0007,32.119576
8,TEST_0008,357.041784
9,TEST_0009,233.858305
